In [13]:
import os
import math
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [14]:
# ==============================================================#
# CONFIG
# ==============================================================#
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/filename.csv"
SAVE_DIR = "/content/drive/MyDrive/Colab Notebooks/result/"
BATCH_SIZE = 64
EPOCHS = 250
LR = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(SAVE_DIR, exist_ok=True)

In [16]:
# ==============================================================#
# FEATURE ENGINEERING FUNCTIONS
# ==============================================================#
def compute_simple_fwi(df):
    tmp = df[['avgtemp_c','avg_humidity','wind_kph','total_precip_mm']].copy()
    for c in tmp.columns:
        mn, mx = tmp[c].min(), tmp[c].max()
        if mx - mn <= 0:
            tmp[c] = 0.5
        else:
            tmp[c] = (tmp[c] - mn) / (mx - mn)
    fwi = 0.45 * tmp['avgtemp_c'] + 0.3 * (1 - tmp['avg_humidity']) + 0.2 * tmp['wind_kph'] - 0.05 * tmp['total_precip_mm']
    return fwi.clip(0, 1)

In [17]:
# ==============================================================#
# DATA PREPROCESSING
# ==============================================================#
df = pd.read_csv(DATA_PATH)
df['acq_date'] = pd.to_datetime(df['acq_date'])
df = df.sort_values('acq_date')

# Group by spatial cell (rounded lat/lon)
df['lat_r'] = df['latitude'].round(3)
df['lon_r'] = df['longitude'].round(3)
df['cell_id'] = df['lat_r'].astype(str) + "_" + df['lon_r'].astype(str)

# Compute derived features
df['fwi_simple'] = compute_simple_fwi(df)
df['dayofyear'] = df['acq_date'].dt.dayofyear
df['hour'] = df['acq_date'].dt.hour
df = df.sort_values(['cell_id','acq_date'])

# Rolling mean features per cell
df[['temp_roll3','precip_roll3','hum_roll3','wind_roll3']] = df.groupby('cell_id')[['avgtemp_c','total_precip_mm','avg_humidity','wind_kph']].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)

# Drop missing rows
df = df.dropna(subset=['avgtemp_c','total_precip_mm','avg_humidity','wind_kph','fwi_simple'])

# Target
df['risk_now'] = df['fwi_simple']  # immediate risk
FEATURE_COLS = ['avgtemp_c','total_precip_mm','avg_humidity','pressure_in','wind_kph',
                'fwi_simple','temp_roll3','precip_roll3','hum_roll3','wind_roll3','dayofyear','hour']

In [18]:
print(df.shape)
print(df[FEATURE_COLS].shape)
print(df.head())


(18191, 19)
(18191, 12)
      latitude  longitude   acq_date  avgtemp_c  total_precip_mm  \
259    10.1522    76.3808 2024-10-24       26.2            22.81   
802    10.1519    76.3924 2024-10-29       26.9             0.60   
262    10.1801    77.8067 2024-10-24       22.9             2.02   
4334   10.4378    78.0548 2024-11-09       24.6             0.10   
4338   10.4395    78.0435 2024-11-09       24.6             0.10   

      avg_humidity  pressure_in  wind_kph   lat_r   lon_r        cell_id  \
259           85.0    29.837917      17.6  10.152  76.381  10.152_76.381   
802           76.0    29.807917      14.8  10.152  76.392  10.152_76.392   
262           86.0    29.862083       6.8  10.180  77.807   10.18_77.807   
4334          79.0    29.862500      11.2  10.438  78.055  10.438_78.055   
4338          79.0    29.862500      11.2  10.440  78.044   10.44_78.044   

      fwi_simple  dayofyear  hour  temp_roll3  precip_roll3  hum_roll3  \
259     0.463314        298     0   

In [19]:
# ==============================================================#
# FEATURE SCALING
# ==============================================================#
scaler = StandardScaler()
df[FEATURE_COLS] = scaler.fit_transform(df[FEATURE_COLS])
joblib.dump({'scaler': scaler}, os.path.join(SAVE_DIR, "scaler.pkl"))
print("✅ StandardScaler fitted and saved.")

✅ StandardScaler fitted and saved.


In [20]:
# ==============================================================#
# DATASET CLASS
# ==============================================================#
class WildfireDataset(Dataset):
    def __init__(self, df, feature_cols):
        self.X = df[feature_cols].values.astype(np.float32)
        self.y = df['risk_now'].values.astype(np.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [21]:
# ==============================================================#
# SPLIT DATA (BY TIME)
# ==============================================================#
t1 = df['acq_date'].quantile(0.7)
t2 = df['acq_date'].quantile(0.85)
train_df = df[df['acq_date'] <= t1]
val_df   = df[(df['acq_date'] > t1) & (df['acq_date'] <= t2)]
test_df  = df[df['acq_date'] > t2]

train_ds = WildfireDataset(train_df, FEATURE_COLS)
val_ds   = WildfireDataset(val_df, FEATURE_COLS)
test_ds  = WildfireDataset(test_df, FEATURE_COLS)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Data points — Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}")

Data points — Train: 12958, Val: 2948, Test: 2285


In [22]:
# ==============================================================#
# MODEL DEFINITION
# ==============================================================#
class RiskModel(nn.Module):
    def __init__(self, n_features, hidden=128, layers=2, dropout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(n_features, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden//2, 1)
        )
    def forward(self, x):
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [26]:
# ==============================================================#
# TRAINING
# ==============================================================#
model = RiskModel(len(FEATURE_COLS)).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

from sklearn.metrics import r2_score

best_val = float('inf')
for epoch in range(EPOCHS):
    # Train
    model.train()
    train_losses = []
    y_train_true, y_train_pred = [], []
    for X, y in train_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        preds = model(X)
        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        y_train_true += y.cpu().tolist()
        y_train_pred += preds.detach().cpu().tolist()

    # Validate
    model.eval()
    val_losses = []
    y_val_true, y_val_pred = [], []
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            preds = model(X)
            loss = criterion(preds, y)
            val_losses.append(loss.item())
            y_val_true += y.cpu().tolist()
            y_val_pred += preds.cpu().tolist()

    # Compute metrics
    mean_train, mean_val = np.mean(train_losses), np.mean(val_losses)
    rmse_train = math.sqrt(mean_squared_error(y_train_true, y_train_pred))
    mae_train = mean_absolute_error(y_train_true, y_train_pred)
    r2_train = r2_score(y_train_true, y_train_pred)

    rmse_val = math.sqrt(mean_squared_error(y_val_true, y_val_pred))
    mae_val = mean_absolute_error(y_val_true, y_val_pred)
    r2_val = r2_score(y_val_true, y_val_pred)

    scheduler.step(mean_val)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss={mean_train:.5f}, RMSE={rmse_train:.4f}, MAE={mae_train:.4f} "
          f"| Val Loss={mean_val:.5f}, RMSE={rmse_val:.4f}, MAE={mae_val:.4f}")

    if mean_val < best_val:
        best_val = mean_val
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, "best_model.pth"))
        print(f"💾 Saved best model (Val Loss={best_val:.5f})")

torch.save(model.state_dict(), os.path.join(SAVE_DIR, "epoch250_model.pth"))


Epoch 1/250 | Train Loss=0.00063, RMSE=0.0252, MAE=0.0160 | Val Loss=0.00009, RMSE=0.0086, MAE=0.0069
💾 Saved best model (Val Loss=0.00009)
Epoch 2/250 | Train Loss=0.00019, RMSE=0.0138, MAE=0.0099 | Val Loss=0.00003, RMSE=0.0058, MAE=0.0044
💾 Saved best model (Val Loss=0.00003)
Epoch 3/250 | Train Loss=0.00015, RMSE=0.0123, MAE=0.0086 | Val Loss=0.00006, RMSE=0.0071, MAE=0.0053
Epoch 4/250 | Train Loss=0.00013, RMSE=0.0113, MAE=0.0080 | Val Loss=0.00005, RMSE=0.0064, MAE=0.0047
Epoch 5/250 | Train Loss=0.00012, RMSE=0.0107, MAE=0.0075 | Val Loss=0.00002, RMSE=0.0044, MAE=0.0032
💾 Saved best model (Val Loss=0.00002)
Epoch 6/250 | Train Loss=0.00011, RMSE=0.0104, MAE=0.0072 | Val Loss=0.00002, RMSE=0.0045, MAE=0.0035
Epoch 7/250 | Train Loss=0.00010, RMSE=0.0101, MAE=0.0070 | Val Loss=0.00001, RMSE=0.0037, MAE=0.0029
💾 Saved best model (Val Loss=0.00001)
Epoch 8/250 | Train Loss=0.00009, RMSE=0.0097, MAE=0.0068 | Val Loss=0.00003, RMSE=0.0049, MAE=0.0036
Epoch 9/250 | Train Loss=0.00009

In [29]:
# ==============================================================#
# TEST EVALUATION
# ==============================================================#
model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "best_model.pth")))
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        preds = model(X)
        y_true += y.cpu().tolist()
        y_pred += preds.cpu().tolist()

rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
print(f"Test: RMSE={rmse:.4f}, MAE={mae:.4f}")
print("✅ Training complete. Model and scaler saved in:", SAVE_DIR)

Test: RMSE=0.0035, MAE=0.0025
✅ Training complete. Model and scaler saved in: /content/drive/MyDrive/Colab Notebooks/result/


In [71]:
import os
import joblib
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import folium
from folium.plugins import HeatMap

In [72]:
# ==============================================================#
# CONFIG
# ==============================================================#
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/result/"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FEATURE_COLS = ['avgtemp_c','total_precip_mm','avg_humidity','pressure_in','wind_kph',
                'fwi_simple','temp_roll3','precip_roll3','hum_roll3','wind_roll3','dayofyear','hour']

In [76]:
# ==============================================================#
# UTILITIES
# ==============================================================#
def compute_simple_fwi(df):
    tmp = df[['avgtemp_c','avg_humidity','wind_kph','total_precip_mm']].copy()
    for c in tmp.columns:
        mn, mx = tmp[c].min(), tmp[c].max()
        if mx - mn <= 0:
            tmp[c] = 0.5
        else:
            tmp[c] = (tmp[c] - mn) / (mx - mn)
    fwi = 0.45 * tmp['avgtemp_c'] + 0.3 * (1 - tmp['avg_humidity']) + 0.2 * tmp['wind_kph'] - 0.05 * tmp['total_precip_mm']
    return fwi.clip(0, 1)

class RiskModel(nn.Module):
    def __init__(self, n_features, hidden=128, dropout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(n_features, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden//2, 1)
        )
    def forward(self, x):
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [77]:
# ==============================================================#
# LOAD MODEL + SCALER
# ==============================================================#
scaler = joblib.load(os.path.join(MODEL_DIR, "scaler.pkl"))['scaler']
model = RiskModel(len(FEATURE_COLS))
model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "best_model.pth"), map_location=DEVICE))
model.to(DEVICE).eval()
print("✅ Model and scaler loaded successfully.")

✅ Model and scaler loaded successfully.


In [78]:
# ==============================================================#
# INFERENCE FUNCTION
# ==============================================================#
def predict_fire_risk(df):
    """Predicts immediate fire risk for one or more rows in a DataFrame."""
    df = df.copy()
    df['fwi_simple'] = compute_simple_fwi(df)
    df['dayofyear'] = df['acq_date'].dt.dayofyear
    df['hour'] = df['acq_date'].dt.hour
    df[['temp_roll3','precip_roll3','hum_roll3','wind_roll3']] = df[['avgtemp_c','total_precip_mm','avg_humidity','wind_kph']].rolling(3, min_periods=1).mean()

    df[FEATURE_COLS] = scaler.transform(df[FEATURE_COLS])
    X = torch.tensor(df[FEATURE_COLS].values, dtype=torch.float32).to(DEVICE)
    with torch.no_grad():
        preds = model(X).cpu().numpy()
    return preds


In [81]:
# ==============================================================#
# HEATMAP GENERATION
# ==============================================================#
def generate_heatmap(data, output_html=None):
    """Creates a folium heatmap from DataFrame with columns [latitude, longitude, risk]."""
    if output_html is None:
        output_html = os.path.join(MODEL_DIR, "wildfire_heatmap.html")

    center = [data['latitude'].mean(), data['longitude'].mean()]
    m = folium.Map(location=center, zoom_start=6)
    heat_data = data[['latitude','longitude','risk']].values.tolist()
    HeatMap(heat_data, radius=10, blur=15, max_zoom=10).add_to(m)
    m.save(output_html)
    print("✅ Heatmap saved to:", output_html)





In [104]:
# ==============================================================#
# EXAMPLE USAGE
# ==============================================================#
if __name__ == "__main__":
    # Example input: one row or multiple rows
    example = pd.DataFrame({
        'latitude':[11.77],
        'longitude':[-76.46],
        'acq_date': pd.to_datetime(["2019-02-21 12:00"]),
        'avgtemp_c': np.random.uniform(35),
        'total_precip_mm': np.random.uniform(0),
        'avg_humidity': np.random.uniform(93),
        'pressure_in': np.random.uniform(29.5),
        'wind_kph': np.random.uniform(20.3)
    })

    risks = predict_fire_risk(example)
    example['risk'] = risks
    print("🔥 Predicted risks:\n", example[['latitude','longitude','risk']])

    # Save heatmap in the model directory
    heatmap_path = os.path.join(MODEL_DIR, "wildfire_heatmap.html")
    generate_heatmap(example, output_html=heatmap_path)

🔥 Predicted risks:
    latitude  longitude      risk
0     11.77     -76.46  0.482213
✅ Heatmap saved to: /content/drive/MyDrive/Colab Notebooks/result/wildfire_heatmap.html


In [94]:
import os
import joblib
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import folium
from folium.plugins import HeatMap
from datetime import datetime, timedelta

In [95]:
# ==============================================================#
# CONFIG
# ==============================================================#
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/result/"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FEATURE_COLS = ['avgtemp_c','total_precip_mm','avg_humidity','pressure_in','wind_kph',
                'fwi_simple','temp_roll3','precip_roll3','hum_roll3','wind_roll3','dayofyear','hour']

In [96]:
# ==============================================================#
# UTILITIES
# ==============================================================#
def compute_simple_fwi(row):
    """Compute simplified FWI for a single row (Series)."""
    t = row['avgtemp_c']/50  # normalize temperature roughly
    h = 1 - row['avg_humidity']/100
    w = row['wind_kph']/100
    p = row['total_precip_mm']/100
    fwi = 0.45*t + 0.3*h + 0.2*w - 0.05*p
    return np.clip(fwi, 0, 1)

class RiskModel(nn.Module):
    def __init__(self, n_features, hidden=128, dropout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(n_features, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden//2, 1)
        )
    def forward(self, x):
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [97]:
# ==============================================================#
# LOAD MODEL + SCALER
# ==============================================================#
scaler = joblib.load(os.path.join(MODEL_DIR, "scaler.pkl"))['scaler']
model = RiskModel(len(FEATURE_COLS))
model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "best_model.pth"), map_location=DEVICE))
model.to(DEVICE).eval()
print("✅ Model and scaler loaded successfully.")

✅ Model and scaler loaded successfully.


In [98]:
# ==============================================================#
# PREDICTION FUNCTION
# ==============================================================#
def predict_risk_row(row):
    """Predict wildfire risk for a single row of weather data."""
    df = pd.DataFrame([row])
    df['fwi_simple'] = compute_simple_fwi(df.iloc[0])
    df['dayofyear'] = df['acq_date'].dt.dayofyear
    df['hour'] = df['acq_date'].dt.hour
    df[['temp_roll3','precip_roll3','hum_roll3','wind_roll3']] = df[['avgtemp_c','total_precip_mm','avg_humidity','wind_kph']].rolling(1).mean()
    X = scaler.transform(df[FEATURE_COLS])
    X = torch.tensor(X, dtype=torch.float32).to(DEVICE)
    with torch.no_grad():
        risk = model(X).cpu().item()
    return risk

def generate_risks(forecast_df):
    """Predict risk for multiple locations and return DataFrame with risks."""
    risks = []
    for idx, row in forecast_df.iterrows():
        r = predict_risk_row(row)
        risks.append(r)
    forecast_df['risk'] = risks
    return forecast_df

In [99]:
# ==============================================================#
# HEATMAP FUNCTION
# ==============================================================#
def generate_heatmap(data, output_html=None):
    """Creates a folium heatmap from DataFrame with columns ['latitude','longitude','risk']."""
    if output_html is None:
        output_html = os.path.join(MODEL_DIR, "wildfire_heatmap.html")
    center = [data['latitude'].mean(), data['longitude'].mean()]
    m = folium.Map(location=center, zoom_start=6)
    heat_data = data[['latitude','longitude','risk']].values.tolist()
    HeatMap(heat_data, radius=15, blur=20, max_zoom=10).add_to(m)
    m.save(output_html)
    print("✅ Heatmap saved to:", output_html)

In [101]:
# ==============================================================#
# EXAMPLE USAGE
# ==============================================================#
if __name__ == "__main__":
    # Forecasted weather for 48 hours ahead for multiple locations
    forecast_data = pd.DataFrame({
        'latitude':[34.52, 35.12, 34.85],
        'longitude':[-118.25, -118.50, -118.10],
        'acq_date':[datetime.now() + timedelta(hours=48)]*3,
        'avgtemp_c':[32, 30, 33],
        'total_precip_mm':[0, 1, 0.5],
        'avg_humidity':[40, 50, 35],
        'pressure_in':[29.9, 29.8, 29.7],
        'wind_kph':[15, 10, 20]
    })

    # Generate predicted risks
    forecast_data = generate_risks(forecast_data)
    print("🔥 Predicted 48-hour risks:\n", forecast_data[['latitude','longitude','risk']])

    # Generate Folium heatmap HTML
    heatmap_path = os.path.join(MODEL_DIR, "wildfire_heatmap_48h.html")
    generate_heatmap(forecast_data, output_html=heatmap_path)

🔥 Predicted 48-hour risks:
    latitude  longitude      risk
0     34.52    -118.25  0.544749
1     35.12    -118.50  0.477413
2     34.85    -118.10  0.601462
✅ Heatmap saved to: /content/drive/MyDrive/Colab Notebooks/result/wildfire_heatmap_48h.html
